In [4]:
!apt-get update && apt-get install swig cmake
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install box2d-py tensorflow keras keras-rl2 gym

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,372 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,347 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packag

# Cart Pole

In [5]:
import gym
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Conv2D, Activation
from tensorflow.keras.optimizers.legacy import Adam
import cv2
import warnings
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy, LinearAnnealedPolicy, EpsGreedyQPolicy, BoltzmannQPolicy
from rl.memory import SequentialMemory
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
import gymnasium
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [6]:
enviroment = 'CartPole-v0'
env = gym.make(enviroment)
states = env.observation_space.shape[0]
actions = env.action_space.n

In [7]:

for _ in range(10):
  state = env.reset()
  done = False
  score = 0

  while not done:
    frame = env.render(mode='rgb_array')
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    action = random.choice([0, 1])
    n_state, reward, done, info = env.step(action)
    score+=reward
  print(f"Episode: {_}, Score: {score}")

Episode: 0, Score: 15.0
Episode: 1, Score: 11.0
Episode: 2, Score: 26.0
Episode: 3, Score: 13.0
Episode: 4, Score: 21.0
Episode: 5, Score: 23.0
Episode: 6, Score: 14.0
Episode: 7, Score: 34.0
Episode: 8, Score: 44.0
Episode: 9, Score: 30.0


In [11]:
def building_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape=(1, states)))
  model.add(Dense(240, activation='relu'))
  model.add(Dense(240, activation='relu'))
  model.add(Dense(120, activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(actions, activation='linear'))
  return model

In [12]:
model = building_model(states, actions)

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_5 (Dense)             (None, 240)               1200      
                                                                 
 dense_6 (Dense)             (None, 240)               57840     
                                                                 
 dense_7 (Dense)             (None, 120)               28920     
                                                                 
 dense_8 (Dense)             (None, 60)                7260      
                                                                 
 dense_9 (Dense)             (None, 2)                 122       
                                                                 
Total params: 95,342
Trainable params: 95,342
Non-trai

# Building Agent using Keras-RL

In [14]:
def building_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape=(1, states)))
  model.add(Dense(240, activation='relu'))
  model.add(Dense(240, activation='relu'))
  model.add(Dense(120, activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(actions, activation='linear'))
  return model


def building_agent(model, actions):
  policy = BoltzmannGumbelQPolicy()
  memory = SequentialMemory(limit=50000, window_length=1)
  dqn = DQNAgent(model=model, memory=memory, policy=policy,
                 nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
  return dqn

model = building_model(states, actions)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 dense_10 (Dense)            (None, 240)               1200      
                                                                 
 dense_11 (Dense)            (None, 240)               57840     
                                                                 
 dense_12 (Dense)            (None, 120)               28920     
                                                                 
 dense_13 (Dense)            (None, 60)                7260      
                                                                 
 dense_14 (Dense)            (None, 2)                 122       
                                                                 
Total params: 95,342
Trainable params: 95,342
Non-trai

In [15]:
model = building_model(states, actions)
dqn = building_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 1.0000
83 episodes - episode_reward: 118.506 [8.000, 200.000] - loss: 0.868 - mae: 17.062 - mean_q: 34.344

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 97s 10ms/step - reward: 1.0000
51 episodes - episode_reward: 195.765 [124.000, 200.000] - loss: 3.504 - mae: 31.802 - mean_q: 63.856

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 98s 10ms/step - reward: 1.0000
51 episodes - episode_reward: 197.706 [152.000, 200.000] - loss: 7.848 - mae: 37.727 - mean_q: 75.700

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 1.0000
50 episodes - episode_reward: 198.420 [136.000, 200.000] - loss: 11.200 - mae: 40.393 - mean_q: 81.131

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 100s 10ms/step - reward: 1.000

# Reloading Agent from Memory

In [17]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 198.000, steps: 198
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
199.8


In [18]:
_ = dqn.test(env, nb_episodes=5, visualize=False)
obs = env.reset()
PATH_SAVED = 'CartPole'
output_file = f'{PATH_SAVED}-video.mp4'
n, m = env.render(mode='rgb_array').shape[:2]
frame_size = (m, n)


frame_size = (m, n)
fps = 45.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

while True:
    frame = env.render(mode='rgb_array')
    video_writer.write(frame)

    action = dqn.forward(obs)
    obs, _, done, _ = env.step(action)

    if done:
        break

env.close()
video_writer.release()

Testing for 5 episodes ...
Episode 1: reward: 192.000, steps: 192
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

# Space invaders

In [22]:
# env = gym.make("SpaceInvaders-v4")
# height, width, channels = env.observation_space.shape
# actions = env.action_space.n

In [23]:
# env.unwrapped.get_action_meanings()

In [25]:
env = gymnasium.make("SpaceInvaders-v4")
height, width, channels = env.observation_space.shape
actions = env.action_space.n
env = gymnasium.make('SpaceInvaders-v0')
env.reset()
for _ in range(1, 20+1):
  state = env.reset()
  done = False
  score = 0

  while not done:
    action = random.choice([0,1,2,3,4,5])
    n_state, reward, done, info, _= env.step(action)
    score+=reward
  print(f"Episode: {_} Score: {score}")
env.close()

Episode: {'lives': 0, 'episode_frame_number': 1711, 'frame_number': 1711} Score: 85.0
Episode: {'lives': 0, 'episode_frame_number': 1439, 'frame_number': 3150} Score: 135.0
Episode: {'lives': 0, 'episode_frame_number': 2323, 'frame_number': 5473} Score: 155.0
Episode: {'lives': 0, 'episode_frame_number': 2419, 'frame_number': 7892} Score: 210.0
Episode: {'lives': 0, 'episode_frame_number': 2417, 'frame_number': 10309} Score: 180.0
Episode: {'lives': 0, 'episode_frame_number': 1153, 'frame_number': 11462} Score: 80.0
Episode: {'lives': 0, 'episode_frame_number': 1889, 'frame_number': 13351} Score: 90.0
Episode: {'lives': 0, 'episode_frame_number': 3175, 'frame_number': 16526} Score: 445.0
Episode: {'lives': 0, 'episode_frame_number': 1149, 'frame_number': 17675} Score: 105.0
Episode: {'lives': 0, 'episode_frame_number': 2179, 'frame_number': 19854} Score: 110.0
Episode: {'lives': 0, 'episode_frame_number': 3910, 'frame_number': 23764} Score: 285.0
Episode: {'lives': 0, 'episode_frame_nu

In [ ]:


model = PPO("MlpPolicy",env, verbose=2)
model.learn(total_timesteps=10000)
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=15)
model.save("SpaceInvaders")
print(f"Mean reward: {mean_reward:.2f}")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 776      |
|    ep_rew_mean     | 175      |
| time/              |          |
|    fps             | 240      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 893         |
|    ep_rew_mean          | 232         |
| time/                   |             |
|    fps                  | 210         |
|    iterations           | 2           |
|    time_elapsed         | 19          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009814488 |
|    clip_fraction        | 0.102       |
|    clip_range     

In [26]:

env_optimize = gymnasium.make('SpaceInvaders-v0', render_mode='rgb_array')

class REINFORCE_SPACEINVADERS:
    def __init__(self, *, PPO_model=None, env=None, saved_model_path='/'):
      self.PPO_model = PPO_model
      self.env = env
      self.saved_model_path = saved_model_path

    def build_optimize(self, *, time_steps=10000, n_evals=10, verbose=0, policy_kwargs=[dict(pi=[64, 64], vf=[64, 64])]):
        env = DummyVecEnv([lambda: self.env])
        model = PPO(self.PPO_model, env, verbose=verbose, policy_kwargs={'net_arch':policy_kwargs})
        model.learn(total_timesteps=time_steps)
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=n_evals)
        model.save(self.saved_model_path)
        print(f"Mean reward: {mean_reward:.2f}")

    def test_model(self, *, episodes=10):
          env = self.env
          model = PPO.load(self.saved_model_path, env=env)
          obs = env.reset()
          frame_size = (env.render().shape[1], env.render().shape[0])
          output_file = f'{self.saved_model_path}-video.mp4'
          fourcc = cv2.VideoWriter_fourcc(*'mp4v')
          fps = 30.0
          video_writer = cv2.VideoWriter(output_file, fourcc, fps, frame_size)
          for i in range(10):
              obs, _ = env.reset()
              done = False
              score = 0
              while not done:
                  frame = env.render()
                  action, _ = model.predict(obs)
                  obs, reward, done, _, _ = env.step(action)
                  frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                  video_writer.write(frame_bgr)
                  score += reward
              print('Episode:{} Score:{}'.format(i, score))

          env.close()

REINFOCE = REINFORCE_SPACEINVADERS(PPO_model="MlpPolicy", env=env_optimize, saved_model_path='PPO_model_path')
REINFOCE.build_optimize(time_steps=10000, n_evals=15, verbose=2, policy_kwargs=[dict(pi=[64,128, 128, 64], vf=[64, 128, 128, 64])])
REINFOCE.test_model(episodes=10)

Using cuda device
Wrapping the env in a VecTransposeImage.
-----------------------------
| time/              |      |
|    fps             | 377  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 246         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009119211 |
|    clip_fraction        | 0.0615      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | 0.00489     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.72        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00765    |
|    value_loss           | 15.7        |
-----------------

# LunarLander